In [2]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.layers import LSTM

max_features = 10000
maxlen = 500
batch_size = 32

print('데이터 로딩...')
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=max_features)

print(len(input_train), '훈련 시퀀스')
print(len(input_test), '테스트 시퀀스')

print('시퀀스 패딩 (samples x time)')

input_train = sequence.pad_sequences(input_train, maxlen=maxlen)
input_test = sequence.pad_sequences(input_test, maxlen=maxlen)

데이터 로딩...
25000 훈련 시퀀스
25000 테스트 시퀀스
시퀀스 패딩 (samples x time)


In [8]:
from keras.models import Sequential
from keras.layers import Embedding, Dense

In [10]:
%%time
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])
history = model.fit(input_train, y_train,
                   epochs=10,
                   batch_size=128,
                   validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 67s 3ms/step - loss: 0.5453 - acc: 0.7462 - val_loss: 0.3701 - val_acc: 0.8522
Epoch 2/10
20000/20000 [==============================] - 67s 3ms/step - loss: 0.3161 - acc: 0.8726 - val_loss: 0.3973 - val_acc: 0.8518
Epoch 3/10
20000/20000 [==============================] - 65s 3ms/step - loss: 0.2447 - acc: 0.9058 - val_loss: 0.2920 - val_acc: 0.8798
Epoch 4/10
20000/20000 [==============================] - 64s 3ms/step - loss: 0.2020 - acc: 0.9261 - val_loss: 0.2810 - val_acc: 0.8868
Epoch 5/10
20000/20000 [==============================] - 64s 3ms/step - loss: 0.1852 - acc: 0.9335 - val_loss: 0.3072 - val_acc: 0.8904
Epoch 6/10
20000/20000 [==============================] - 63s 3ms/step - loss: 0.1619 - acc: 0.9412 - val_loss: 0.3821 - val_acc: 0.8838
Epoch 7/10
20000/20000 [==============================] - 64s 3ms/step - loss: 0.1514 - acc: 0.9458 - val_loss: 0.3072 - val_acc:

In [2]:
import os
import tensorflow as tf
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

## plaid ml 을 활용한 맥북 gpu 사용결과

In [8]:
import numpy as np
import os
import time

os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

import keras
import keras.applications as kapp
from keras.datasets import cifar10

(x_train, y_train_cats), (x_test, y_test_cats) = cifar10.load_data()
batch_size = 8
x_train = x_train[:batch_size]
x_train = np.repeat(np.repeat(x_train, 7, axis=1), 7, axis=2)
model = kapp.VGG19()
model.compile(optimizer='sgd', loss='categorical_crossentropy',
              metrics=['accuracy'])

print("Running initial batch (compiling tile program)")
y = model.predict(x=x_train, batch_size=batch_size)

# Now start the clock and run 10 batches
print("Timing inference...")
start = time.time()
for i in range(10):
    y = model.predict(x=x_train, batch_size=batch_size)
print("Ran in {} seconds".format(time.time() - start))

574717952/574710816 [==============================] - 65s 0us/step
Running initial batch (compiling tile program)


INFO:plaidml:Analyzing Ops: 76 of 194 operations complete
INFO:plaidml:Analyzing Ops: 141 of 194 operations complete


Timing inference...
Ran in 3.6385529041290283 seconds


### CPU 사용 결과

In [1]:
import numpy as np
import os
import time

import keras
import keras.applications as kapp
from keras.datasets import cifar10

(x_train, y_train_cats), (x_test, y_test_cats) = cifar10.load_data()
batch_size = 8
x_train = x_train[:batch_size]
x_train = np.repeat(np.repeat(x_train, 7, axis=1), 7, axis=2)
model = kapp.VGG19()
model.compile(optimizer='sgd', loss='categorical_crossentropy',
              metrics=['accuracy'])

print("Running initial batch (compiling tile program)")
y = model.predict(x=x_train, batch_size=batch_size)

# Now start the clock and run 10 batches
print("Timing inference...")
start = time.time()
for i in range(10):
    y = model.predict(x=x_train, batch_size=batch_size)
print("Ran in {} seconds".format(time.time() - start))

Using TensorFlow backend.


Running initial batch (compiling tile program)
Timing inference...
Ran in 18.85858726501465 seconds


In [ ]:
import os
import tensorflow as tf
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"